In [4]:
from cdasws import CdasWs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

## downward and save data

In [7]:
class sc_cdaWeb():
    '''to obtain magnetic field & plasma data from cdaweb'''
    
    def __init__(self, name, tRange,*args):
        self.name = name
        self.time_period = tRange
        self.raw = {} # raw data, stored in dict
        self.attrs = {}
        self.data=pd.DataFrame() # data after interpolation
#         if args !=():
#             self.add_instrument(args)
            
    
    def findData(self,sc_ins):
        instrumt = sc_ins[self.name]
#         raw_data = {}
        
        cdas = CdasWs()
        # i: 仪器数据目录名称，e.g., WI_MAG
        # j: 仪器数据子目录名称, e.g., WI_MAG_BT, WI_MAG_BN
        for i in instrumt.columns:
            tempData = pd.DataFrame()
            for j in instrumt[i].dropna():
                
                temp_data = cdas.get_data(i, [j], self.time_period[0],self.time_period[1],
                                          binData={'interpolateMissingValues': True})[1]
                
                # trim ACE data according to attrs Info
                if self.name == 'Ace':
                    temp_data[j][ temp_data[j]==temp_data[j].attrs['FILLVAL'] ] = np.nan
                    temp_data[j][ (temp_data[j]>temp_data[j].attrs['VALIDMAX'])|(temp_data[j]<temp_data[j].attrs['VALIDMIN']) ] = np.nan

                # Wind instruments' name format change
                if self.name == 'Wind':
                    j = j.replace('.','$')
    
                if len(temp_data[j].shape) == 1:
                    tempData[j] = temp_data[j]
                elif len(temp_data[j].shape) == 2 and temp_data[j].shape[1]==3:
                    tempData[j+'_x'] = np.array(temp_data[j])[:,0]
                    tempData[j+'_y'] = np.array(temp_data[j])[:,1]
                    tempData[j+'_z'] = np.array(temp_data[j])[:,2]
                elif len(temp_data[j].shape) == 2 and self.name == 'Ulysses' and j == 'Density':
                    tempData[j] = np.array(temp_data[j])[:,0]
                else:
                    tempData[j]=np.zeros(temp_data[j].shape[0])
                    print('Failed to get %s_%s'%(i,j))
                
                # recover Wind instruments' name
                if self.name == 'Wind':
                    j = j.replace('$','.')
                
                if j == instrumt[i].dropna().iloc[-1]:
                    if self.name == 'PSP' and j.find('mag_RTN')>=0: # PSP's MAG_RTN data epoch name 
                        tempData['Epoch'] = temp_data['epoch_mag_RTN']
                    else: # other SC instruments
                        tempData['Epoch'] = temp_data['Epoch']
                
                # save instruments' attributes
                self.attrs[j] = temp_data[j].attrs

            # # # 四分位 去除离群值
            # # 四分位法会删除较多数据，not good enough.
            raw = tempData.set_index('Epoch')
            # Q1=raw.quantile(0.25)
            # Q3=raw.quantile(0.75)
            # IQR=Q3-Q1
            # lowqe_bound=Q1 - 2 * IQR
            # upper_bound=Q3 + 2 * IQR
            # IQR_raw = raw[~((raw < lowqe_bound) |(raw > upper_bound)).any(axis=1)]
                
            self.raw[i] = raw # IQR_raw # 将 同一个instrument data构成的 DataFrame存入dict的一个位置。dict 元素数 = number of instrument
            
    def savData(self,path):

        writer = pd.ExcelWriter(path+'%s_%s_%s.xlsx'%(self.name, self.time_period[0],self.time_period[1]), engine='xlsxwriter')

        for key in self.raw:
            self.raw[key].to_excel(writer, sheet_name= key , index = True)
            
        writer.save()

In [21]:
cdas = CdasWs()
temp_data = cdas.get_data('AC_H3_MFI', ['Magnitude'], '2010-02-14 00:00:00Z','2010-02-14 23:59:59Z',
                            dataRepresentation={'interpolateMissingValues': True})

In [28]:
# 1, generate sc instrument dict
# 通过 Series 创建 sc dataframe, 并与其他sc合并保存在字典
# sc_Instrument={'Wind':pd.DataFrame({'WI_PLSP_3DP':pd.Series(['MOM.P.DENSITY','MOM.P.MAGF','MOM.P.VELOCITY'])}),
#                 'Ulysses':pd.DataFrame({'UY_1SEC_VHM':pd.Series(['B_RTN']), 'UY_M0_BAI':pd.Series(['Density','Velocity'])}),
#                 'Ace':pd.DataFrame({'AC_H3_MFI':pd.Series(['BRTN']), 'AC_H0_SWE':pd.Series(['Np','V_RTN'])})}
sc_Instrument={'Ace':pd.DataFrame({'AC_H3_MFI':pd.Series(['Magnitude','BRTN'])})}

# 2, gain data from cdaWeb
tRange = ['2010-02-14 00:00:00Z','2010-02-15 00:00:00Z']
sc = sc_cdaWeb('Ace', tRange)
sc.findData(sc_Instrument)

In [29]:
# 3, interpolation
# freq = 'H':1 hour, '3T':3 min, '5S': 5 sec
# tSeries = [t.timestamp() for t in pd.date_range(start=tRange[0],end = tRange[1], freq='10S')]
# sc.timeRes_interp(tSeries)

# 4
# save data to .csv
path='/home/xiangyu/文档/Space Pyhsics/3 Alfven slow solar wind/CodeinPython/'
sc.savData(path)

## Power spectral analysis

In [147]:
from scipy.fft import fft, ifft

def nextpow2(N):
    ''' same with the nextpow2 in Matlab'''
    return np.ceil(np.log2(N))

def complexArray(row,col):
    '''Create an array to store complex'''
    a = np.zeros((int(row),int(col)),dtype=complex)
    return a

def cart2pol(x, y):
    ''' same as the cart2pol in Matlab'''
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return phi, rho 

def waveletPhiTformula8(time):
    omega = 6

    phiT = (np.pi**-0.25)*(np.exp(1j*omega*time) - np.exp(-0.5*omega**2))*np.exp(-0.5*time**2)
    # phiT = np.power(np.pi, -0.25)*np.exp(1j*omega*time) - np.exp(-0.5*np.power(omega, 2))*np.exp(-0.5*np.power(time,2))

    return phiT

def waveletFunc(data, timeInterval, deltaT, M):

    tempData = data.loc[timeInterval[0]:timeInterval[1]] # partical data
    waveletData = pd.DataFrame(columns=['frequency'])

    omega = 6
    C = 1.06
    N = len(tempData)
    L = int(np.power(2, nextpow2( np.power(2,nextpow2(N)) +1)))

    time_gn_temp = np.arange(-np.floor(N/2)+1, np.floor(N/2)+1, 1)*deltaT # np.arange 不包含最后一个点，因此终止点+1 (从而与Matlab一致).
    time_gn = time_gn_temp[1:]

    S0 = 4*deltaT

    kMaxWaveletScale = 0.25
    sMax = kMaxWaveletScale*deltaT*N

    sM = np.logspace(np.log10(S0), np.log10(sMax), M)
    frequency = (1+1/(2*omega**2))*omega/(2*np.pi*sM)
    k17 = 4*np.pi/(C*omega*N)
    waveletData['frequency'] = frequency

    for col in tempData:
        gN18Gk = complexArray(M,L)
        fkBr = complexArray(M,L)
        fStn13Br = complexArray(M,L)

        for i in range(M):
            gN18Gk[i,:] = fft( np.conj( waveletPhiTformula8(-time_gn/sM[i]) ), L)
            fkBr[i,:] = fft(tempData[col].to_numpy(), L)
            fStn13Br[i,:] = (deltaT*sM[i]**-0.5)*ifft(gN18Gk[i,:]*fkBr[i,:])
        
        fStn13BrFinal = fStn13Br[:,int(np.floor(N/2)-1):int(np.floor(N*3/2-1))] # 起始点-1. 因为python index从0开始，matlab 从1开始。
        powerR = np.abs(fStn13BrFinal)**2
        powerAll = k17*powerR
        globalWs = np.sum(powerAll,axis=1)

        waveletData['pow%s'%col] = globalWs # power/Hz of input signal
    

    # frequency, globalWs, powerAll
    return waveletData

def decomposeBrtn(df, timeInterval):
    '''decompose Bt Bn to Balpha Bbeta'''
    # Brab: Br, B_alpha, B_beta, in specific time interval.
    Brab = pd.DataFrame(df.loc[timeInterval[0]:timeInterval[1], ['BRTN_x','BRTN_y','BRTN_z']])

    theta, _= cart2pol(Brab['BRTN_y'],Brab['BRTN_z'])
    thetaB0toBt = np.rad2deg(theta) # obtain degrees

    thetaB0toBt180 = thetaB0toBt.copy()
    thetaB0toBt360 = thetaB0toBt180.copy() # range from -180 to 180
    thetaB0toBt360[thetaB0toBt360<0] = thetaB0toBt360[thetaB0toBt360<0]+360

    thetaBetatoBt = thetaB0toBt180.mean()

    # decompose
    Bbeta = Brab['BRTN_y']*np.cos(np.deg2rad(thetaBetatoBt)) + Brab['BRTN_z']*np.sin(np.deg2rad(thetaBetatoBt))
    Balpha = Brab['BRTN_y']*np.sin(np.deg2rad(thetaBetatoBt)) - Brab['BRTN_z']*np.cos(np.deg2rad(thetaBetatoBt))
    Brab['Balpha'] = Balpha
    Brab['Bbeta'] = Bbeta

    # also save to original data
    df.loc[timeInterval[0]:timeInterval[1],'Balpha'] = Balpha
    df.loc[timeInterval[0]:timeInterval[1],'Bbeta'] = Bbeta
    

    angRange = 1
    if not angRange:
        angMin = 0
        angMax = 360
        thetaB0toBt = thetaB0toBt360
    else:
        angMin = -180
        angMax = 180
        thetaB0toBt = thetaB0toBt180
    
    return {'Brab':Brab,'thetaB0toBt':thetaB0toBt, 'angMin':angMin,'angMax':angMax}

def plotPSD(dataPsd, cols=['powMagnitude', 'powBRTN_x', 'powBRTN_y', 'powBRTN_z'], legend=['$B_{mag}$','$B_r$','$B_t$','$B_n$'], ifsave = False,figname = 'PSD'):
    ''' plot PSD.
        cols - legend pairs:
        (1) ['powMagnitude', 'powBRTN_x', 'powBRTN_y', 'powBRTN_z'] -- ['$B_{mag}$','$B_r$','$B_t$','$B_n$']
        (2) ['powMagnitude', 'powBRTN_x', 'powBalpha', 'powBbeta'] -- ['$B_{mag}$','$B_r$','$B_{\\alpha}$','$B_{\\beta}$']
    '''

    fig,ax = plt.subplots(1,figsize=(7,6))

    for i,j in zip(cols, legend):
        ax.plot(dataPsd['frequency'], dataPsd[i], label=j, linewidth=3)


    ax.legend(fontsize = 13)
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylabel('PSD (Power/Hz)',fontsize=14)
    ax.set_xlabel('Frequency (Hz)',fontsize=14)
    fig.tight_layout()
    
    if ifsave:
        fig.savefig('./'+ figname +'.png', dpi=300)

In [71]:
# 0. Read saved data
eventName = 'Ace_2010-02-14 00:00:00Z_2010-02-15 00:00:00Z.xlsx'
path='/home/xiangyu/文档/Space Pyhsics/3 Alfven slow solar wind/CodeinPython/'

df = pd.read_excel(path+eventName,index_col=0)
df.index = df.index.to_numpy().astype('datetime64[s]') # remove microseconds in time index

,Magnitude,BRTN_x,BRTN_y,BRTN_z
2010-02-14 00:00:00,5.818,-0.037,5.690,1.213
2010-02-14 00:00:01,5.810,-0.138,5.675,1.235
2010-02-14 00:00:02,5.859,-0.007,5.726,1.242
2010-02-14 00:00:03,5.880,0.031,5.736,1.293
2010-02-14 00:00:04,5.827,-0.006,5.687,1.272
...,...,...,...,...
2010-02-14 23:59:55,8.659,0.257,6.956,-5.150
2010-02-14 23:59:56,8.715,0.235,7.026,-5.150
2010-02-14 23:59:57,8.646,0.216,7.006,-5.062
2010-02-14 23:59:58,8.633,0.251,6.951,-5.114


In [83]:
# 1
timeInterval = ['2010-02-14 10:00:00','2010-02-14 19:59:59']
M = 150
deltaT = 1

# 2. Decompose Bt Bn to Balpha Bbeta
thetaDict = decomposeBrtn(df, timeInterval)

# 3. Perform spectral analysis on all B. Return a DataFrame.
waveletData = waveletFunc(df, timeInterval, deltaT, M) # 结果与 Matlab相同

# 4 plot PSD
savefig = False

# Brtn
plotPSD(waveletData, figname='%s_Brtn'%eventName[0:15], ifsave=savefig)

# Brab
# plotPSD(waveletData, figname='%s_Brab'%eventName[0:15], cols=['powMagnitude', 'powBRTN_x', 'powBalpha', 'powBbeta'], legend=['$B_{mag}$','$B_r$','$B_{\\alpha}$','$B_{\\beta}$'],ifsave=savefig)